In [213]:
import re
import pandas as pd

In [214]:
property_files = '../data/raw/property_meta'

In [215]:
df = pd.read_csv(f'{property_files}/property_metadata.csv')

In [216]:
df.head()

,url,name,type,cost_text,coordinates,rooms,desc
0,https://www.domain.com.au/5-1-3-rex-avenue-alp...,5/1-3 Rex Avenue Alphington VIC 3078,Apartment / Unit / Flat,$440.00,"[-37.7856068, 145.0249141]","[['2 Beds'], ['1 Bath'], ['1 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."
1,https://www.domain.com.au/805-81-queens-road-m...,805/81 Queens Road Melbourne VIC 3004,Studio,$380 weekly,"[-37.853804, 144.9798785]","[['0 Beds'], ['1 Bath'], ['1 Parking']]",Leasing Concierge
2,https://www.domain.com.au/12-1-rex-avenue-alph...,12/1 Rex Avenue Alphington VIC 3078,Apartment / Unit / Flat,$460.00,"[-37.7856092, 145.0249185]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Pyper Byrne
3,https://www.domain.com.au/3-lime-ct-bellfield-...,3 Lime Ct Bellfield VIC 3081,House,$425 PW | Weston Real Estate,"[-37.7548292, 145.041952]","[['3 Beds'], ['1 Bath'], ['3 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."
4,https://www.domain.com.au/24-36-ridley-street-...,24/36 Ridley Street Albion VIC 3020,Apartment / Unit / Flat,$250,"[-37.7834829, 144.8233623]","[['1 Bed'], ['1 Bath'], ['1 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."


In [217]:
df['postcode'] = df['name'].str[-4:]
df['name'] = df['name'].str[:-5]

In [226]:
rooms = df['rooms'].str.split(',', expand=True)
rooms = rooms.iloc[:, :-1]
rooms.columns = ['bed', 'bath', 'car']
rooms.fillna(0)
rooms['bed'] = rooms.bed.str.extract('(\d+)').fillna(0).astype('int64')
rooms['bath'] = rooms.bath.str.extract('(\d+)').fillna(0).astype('int64')
rooms['car'] = rooms.car.str.extract('(\d+)').fillna(0).astype('int64')

In [227]:
rooms.head()

,bed,bath,car
0,2,1,1
1,0,1,1
2,2,1,1
3,3,1,3
4,1,1,1


In [229]:
df = pd.concat([df, rooms], axis=1)
df = df.drop(columns=['rooms'])

In [230]:
df.head()

,url,name,type,cost_text,coordinates,desc,postcode,bed,bath,car
0,https://www.domain.com.au/5-1-3-rex-avenue-alp...,5/1-3 Rex Avenue Alphington VIC,Apartment / Unit / Flat,$440.00,"[-37.7856068, 145.0249141]","class=""css-dxogle"">* Unverified feature<svg a...",3078,2,1,1
1,https://www.domain.com.au/805-81-queens-road-m...,805/81 Queens Road Melbourne VIC,Studio,$380 weekly,"[-37.853804, 144.9798785]",Leasing Concierge,3004,0,1,1
2,https://www.domain.com.au/12-1-rex-avenue-alph...,12/1 Rex Avenue Alphington VIC,Apartment / Unit / Flat,$460.00,"[-37.7856092, 145.0249185]",Pyper Byrne,3078,2,1,1
3,https://www.domain.com.au/3-lime-ct-bellfield-...,3 Lime Ct Bellfield VIC,House,$425 PW | Weston Real Estate,"[-37.7548292, 145.041952]","class=""css-dxogle"">* Unverified feature<svg a...",3081,3,1,3
4,https://www.domain.com.au/24-36-ridley-street-...,24/36 Ridley Street Albion VIC,Apartment / Unit / Flat,$250,"[-37.7834829, 144.8233623]","class=""css-dxogle"">* Unverified feature<svg a...",3020,1,1,1


In [243]:
from collections import Counter

In [244]:
Counter(" ".join(df["desc"].str.lower()).split()).most_common(100)

TypeError: sequence item 4207: expected str instance, float found

In [ ]:
Counter(" ".join(df["desc"]).split()).most_common(100)

In [231]:
df.dtypes

url            object
name           object
type           object
cost_text      object
coordinates    object
desc           object
postcode       object
bed             int64
bath            int64
car             int64
dtype: object

In [234]:
df['postcode'].value_counts()

3000    223
3141    138
3182    123
3121    115
3181    110
       ... 
3913      1
3713      1
3090      1
3804      1
3036      1
Name: postcode, Length: 189, dtype: int64

In [233]:
df['type'].value_counts()

Apartment / Unit / Flat          2560
House                            2194
Townhouse                         581
Studio                            154
Villa                              16
Carspace                            8
Semi-Detached                       5
Acreage / Semi-Rural                5
Terrace                             4
Rural                               1
Penthouse                           1
New Apartments / Off the Plan       1
Duplex                              1
Name: type, dtype: int64